In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from numba import jit

In [3]:
pd.options.display.max_columns = 50

### Path

In [4]:
path = Path('/kaggle/m5_forecasting/')
assert(path.exists())

In [5]:
CAL_DTYPES={"event_name_2": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [6]:
h = 28 
max_lags = h * 2 + 1
# max_lags = 57
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [7]:
def prepare_tables():
    prices = pd.read_csv(path/"sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv(path/"calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    return prices, cal

In [8]:
%%time

prices, cal = prepare_tables()

CPU times: user 1.59 s, sys: 219 ms, total: 1.8 s
Wall time: 1.99 s


In [9]:
def create_event_map(field):
    return {v: k for k, v in enumerate(cal[field].unique())}

In [10]:
def convert_to_type(df, cols, dt_type):
    for type_name in cols:
        df[type_name] = df[type_name].astype(dt_type)

def convert_uint8(df, cols):
    convert_to_type(df, cols, "uint8")
    
def convert_float16(df, cols):
    convert_to_type(df, cols, "float16")

In [11]:
def replace_cal_cols():
    event_name_1_map = create_event_map('event_name_1')
    cal.replace({'event_name_1': event_name_1_map}, inplace=True)
    event_type_1_map = create_event_map('event_type_1')
    cal.replace({'event_type_1': event_type_1_map}, inplace=True)
    return event_name_1_map, event_type_1_map

event_name_1_map, event_type_1_map = replace_cal_cols()

In [12]:
cal[(cal.date > '2012-01-01') & (cal.date < '2012-01-05')]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
338,2012-01-02,11149,1,3,1,2012,d_339,0,0,0,0,1.0,0.0,1.0
339,2012-01-03,11149,5,4,1,2012,d_340,0,0,0,0,1.0,1.0,1.0
340,2012-01-04,11149,6,5,1,2012,d_341,0,0,0,0,1.0,0.0,0.0


In [13]:
uint8_types= ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'month', 'wday', 'weekday', 
              'snap_CA', 'snap_TX', 'snap_WI']
convert_uint8(cal, uint8_types)

In [14]:
def get_elapsed(dt, event_name='Christmas', col='event_name_1', event_map=event_name_1_map, before=False):
    dt.sort_values(['date'], ascending=[(not before)], inplace=True)
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    res = []
    event = event_map[event_name]
    for v,d in zip(dt[col].values, dt.date.values):
        if v == event:
            last_date = d
        elapsed = ((d-last_date).astype('timedelta64[D]') / day1)
        res.append(elapsed)
    field_name = f"{'before' if before else 'after'}_{event_name.lower().replace(' ', '_')}"
    dt[field_name] = res
    dt[field_name] = dt[field_name].fillna(0)
    dt[field_name] = dt[field_name].astype('int16')

In [15]:
# ellapsed_fields = ['Christmas', 'Easter', 'Ramadan starts']
# for f in ellapsed_fields:
#     get_elapsed(cal, f, 'event_name_1', event_name_1_map, False)

In [16]:
def add_mean_over_period(dt, period='weekday'):
    df_sales_id_period = dt[['id', 'sales', period]].groupby(['id', period])[['sales']].mean().reset_index()
    df_sales_id_period['sales'] = df_sales_id_period['sales'].astype('float16')
    df_sales_id_period.rename(columns={'sales': f'sales_by_{period}'}, inplace=True)
    return dt.merge(df_sales_id_period, on=['id', period], how='left')

In [17]:
day_of_year = 'Dayofyear'

def prepare_day_of_year(df):
    df[day_of_year] = getattr(df['date'].dt, day_of_year.lower()).astype('uint16')

In [18]:
def add_days_before(dt, day=25, month=12, col_name='before_christmas'):
    diff_list = []
    for d in cal['date']:
        target = datetime(d.year, month, day)
        diff = (target - d.to_pydatetime()).days
        if(diff < 0):
            christmas = datetime(d.year + 1, 12, 25)
            diff = (target - d.to_pydatetime()).days
        diff_list.append(diff)
    dt[col_name] = diff_list
    dt[col_name] = dt[col_name].astype('uint16')

In [19]:
add_days_before(cal)
# add_days_before(cal, day=31, month=10, col_name='before_halloween')

In [20]:
easter_western = {2011: datetime(2011, 4, 24), 2012: datetime(2012, 4, 8), 2013: datetime(2013, 3, 31)
 , 2014: datetime(2014, 4, 20), 2015: datetime(2015, 4, 5), 2016: datetime(2016, 4, 27), 2017: datetime(2017, 4, 16)}

def add_days_before_special_date(dt, date_dict=easter_western, col_name='before_easter'):
    diff_list = []
    for d in cal['date']:
        year = d.year
        target = date_dict[year]
        diff = (target - d.to_pydatetime()).days
        if(diff < 0):
            target = date_dict[year + 1]
            diff = (target - d.to_pydatetime()).days
        diff_list.append(diff)
    dt[col_name] = diff_list
    dt[col_name] = dt[col_name].astype('uint16')

In [21]:
# add_days_before_special_date(cal)

In [22]:
cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              1969 non-null   datetime64[ns]
 1   wm_yr_wk          1969 non-null   int16         
 2   weekday           1969 non-null   uint8         
 3   wday              1969 non-null   uint8         
 4   month             1969 non-null   uint8         
 5   year              1969 non-null   int16         
 6   d                 1969 non-null   object        
 7   event_name_1      1969 non-null   uint8         
 8   event_type_1      1969 non-null   uint8         
 9   event_name_2      1969 non-null   uint8         
 10  event_type_2      1969 non-null   uint8         
 11  snap_CA           1969 non-null   uint8         
 12  snap_TX           1969 non-null   uint8         
 13  snap_WI           1969 non-null   uint8         
 14  before_christmas  1969 n

In [23]:
cal

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,before_christmas
0,2011-01-29,11101,2,1,1,2011,d_1,0,0,0,0,0,0,0,330
1,2011-01-30,11101,3,2,1,2011,d_2,0,0,0,0,0,0,0,329
2,2011-01-31,11101,1,3,1,2011,d_3,0,0,0,0,0,0,0,328
3,2011-02-01,11101,5,4,2,2011,d_4,0,0,0,0,1,1,0,327
4,2011-02-02,11101,6,5,2,2011,d_5,0,0,0,0,1,0,1,326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,6,5,6,2016,d_1965,0,0,0,0,0,1,1,193
1965,2016-06-16,11620,4,6,6,2016,d_1966,0,0,0,0,0,0,0,192
1966,2016-06-17,11620,0,7,6,2016,d_1967,0,0,0,0,0,0,0,191
1967,2016-06-18,11621,2,1,6,2016,d_1968,0,0,0,0,0,0,0,190


In [24]:
state_map = {'CA': 0, 'TX': 1, 'WI': 2}

In [25]:
def process_snap(dt):
    dt['snap'] = ((dt['snap_CA'] == 1) & (dt['state_id'] == state_map['CA']) 
              | (dt['snap_TX'] == 1) & (dt['state_id'] == state_map['TX']) 
              | (dt['snap_WI'] == 1) & (dt['state_id'] == state_map['WI']))
    
    dt['snap'] = dt['snap'].astype('uint8')
    
    for c in ['snap_CA', 'snap_TX', 'snap_WI']:
        del dt[c]

In [26]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    
    start_day = max(1, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    dt = pd.read_csv(path/"sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols + ['state_id'], dtype = dtype)
    
    dt.replace({'state_id': state_map}, inplace=True)
    dt['state_id'] = dt['state_id'].astype("int16")
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols + ['state_id'],
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    dt.sort_values(['id', 'date'], inplace=True)
    prepare_day_of_year(dt)
    
    ## Dates
    
    date_features = {
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "year": "year",
        "mday": "day",
    }
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")
            
    uint8_types= ['month', 'wday', 'mday', 'week']
    convert_uint8(dt, uint8_types)
    
    ## Lag Price
    
    dt[f'lag_price_1'] = dt[["id","sell_price"]].groupby("id")["sell_price"].shift(1)
    for f in ['lag_price_1', 'sell_price']:
        dt[f] = dt[f].astype('float16')
    
    dt['sales'] = dt['sales'].astype('float16')
    
    return dt

In [27]:
FIRST_DAY = 1400 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [28]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)

CPU times: user 15.8 s, sys: 2.03 s, total: 17.9 s
Wall time: 18 s


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15537606 entries, 1572 to 15534731
Data columns (total 27 columns):
 #   Column            Dtype         
---  ------            -----         
 0   id                object        
 1   item_id           int16         
 2   dept_id           int16         
 3   store_id          int16         
 4   cat_id            int16         
 5   state_id          int16         
 6   d                 object        
 7   sales             float16       
 8   date              datetime64[ns]
 9   wm_yr_wk          int16         
 10  weekday           uint8         
 11  wday              uint8         
 12  month             uint8         
 13  year              int16         
 14  event_name_1      uint8         
 15  event_type_1      uint8         
 16  event_name_2      uint8         
 17  event_type_2      uint8         
 18  snap_CA           uint8         
 19  snap_TX           uint8         
 20  snap_WI           uint8         
 21  bef

In [30]:
df

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,before_christmas,sell_price,Dayofyear,week,mday,lag_price_1
1572,FOODS_1_001_CA_1_validation,1536,4,0,2,0,d_1400,0.0,2014-11-28,11443,0,7,11,2014,0,0,0,0,0,0,0,27,5.320312,332,48,28,NaN
40380,FOODS_1_001_CA_1_validation,1536,4,0,2,0,d_1401,0.0,2014-11-29,11444,2,1,11,2014,0,0,0,0,0,0,0,26,5.320312,333,48,29,5.320312
40381,FOODS_1_001_CA_1_validation,1536,4,0,2,0,d_1402,0.0,2014-11-30,11444,3,2,11,2014,0,0,0,0,0,0,0,25,5.320312,334,48,30,5.320312
40382,FOODS_1_001_CA_1_validation,1536,4,0,2,0,d_1403,0.0,2014-12-01,11444,1,3,12,2014,0,0,0,0,1,1,0,24,5.320312,335,49,1,5.320312
40383,FOODS_1_001_CA_1_validation,1536,4,0,2,0,d_1404,0.0,2014-12-02,11444,5,4,12,2014,0,0,0,0,1,0,1,23,5.320312,336,49,2,5.320312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15466564,HOUSEHOLD_2_516_WI_3_validation,2047,3,9,1,2,d_1909,0.0,2016-04-20,11612,6,5,4,2016,0,0,0,0,0,0,0,249,1.000000,111,16,20,1.000000
15466565,HOUSEHOLD_2_516_WI_3_validation,2047,3,9,1,2,d_1910,0.0,2016-04-21,11612,4,6,4,2016,0,0,0,0,0,0,0,248,1.000000,112,16,21,1.000000
15466566,HOUSEHOLD_2_516_WI_3_validation,2047,3,9,1,2,d_1911,0.0,2016-04-22,11612,0,7,4,2016,0,0,0,0,0,0,0,247,1.000000,113,16,22,1.000000
15534730,HOUSEHOLD_2_516_WI_3_validation,2047,3,9,1,2,d_1912,0.0,2016-04-23,11613,2,1,4,2016,0,0,0,0,0,0,0,246,1.000000,114,16,23,1.000000


In [31]:
def create_fea(dt):
    
    wins = [7, 28]
    lags = [7, 28]
    
    for win in wins:
        mean_col = f'mean_{win}'
        df[mean_col] = df[["id","sales"]].groupby("id")["sales"].transform(lambda x : x.rolling(win).mean())
        print(mean_col)
        for lag in lags:
            df[f'rmean_{win}_{lag}'] = df[["id", mean_col]].groupby("id").shift(lag)
            print(f'rmean_{win}_{lag}')
        del dt[mean_col]
            
    ra = range(1, 3)
    for simple_lag in ra:
        dt[f'lag_{simple_lag}'] = dt[["id","sales"]].groupby("id")["sales"].shift(simple_lag)
        
    print('simple_lags', ra)

In [32]:
%%time

create_fea(df)

mean_7
rmean_7_7
rmean_7_28
mean_28
rmean_28_7
rmean_28_28
simple_lags range(1, 3)
CPU times: user 38.3 s, sys: 1.73 s, total: 40.1 s
Wall time: 40.1 s


In [33]:
df.dropna(inplace = True)
df.shape

(13860656, 33)

In [34]:
df.tail()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,before_christmas,sell_price,Dayofyear,week,mday,lag_price_1,rmean_7_7,rmean_7_28,rmean_28_7,rmean_28_28,lag_1,lag_2
15466564,HOUSEHOLD_2_516_WI_3_validation,2047,3,9,1,2,d_1909,0.0,2016-04-20,11612,6,5,4,2016,0,0,0,0,0,0,0,249,1.0,111,16,20,1.0,0.0,0.0,0.0,0.107117,0.0,0.0
15466565,HOUSEHOLD_2_516_WI_3_validation,2047,3,9,1,2,d_1910,0.0,2016-04-21,11612,4,6,4,2016,0,0,0,0,0,0,0,248,1.0,112,16,21,1.0,0.0,0.0,0.0,0.107117,0.0,0.0
15466566,HOUSEHOLD_2_516_WI_3_validation,2047,3,9,1,2,d_1911,0.0,2016-04-22,11612,0,7,4,2016,0,0,0,0,0,0,0,247,1.0,113,16,22,1.0,0.0,0.0,0.0,0.107117,0.0,0.0
15534730,HOUSEHOLD_2_516_WI_3_validation,2047,3,9,1,2,d_1912,0.0,2016-04-23,11613,2,1,4,2016,0,0,0,0,0,0,0,246,1.0,114,16,23,1.0,0.0,0.0,0.0,0.107117,0.0,0.0
15534731,HOUSEHOLD_2_516_WI_3_validation,2047,3,9,1,2,d_1913,0.0,2016-04-24,11613,3,2,4,2016,0,0,0,0,0,0,0,245,1.0,115,16,24,1.0,0.0,0.0,0.0,0.071411,0.0,0.0


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13860656 entries, 1487908 to 15534731
Data columns (total 33 columns):
 #   Column            Dtype         
---  ------            -----         
 0   id                object        
 1   item_id           int16         
 2   dept_id           int16         
 3   store_id          int16         
 4   cat_id            int16         
 5   state_id          int16         
 6   d                 object        
 7   sales             float16       
 8   date              datetime64[ns]
 9   wm_yr_wk          int16         
 10  weekday           uint8         
 11  wday              uint8         
 12  month             uint8         
 13  year              int16         
 14  event_name_1      uint8         
 15  event_type_1      uint8         
 16  event_name_2      uint8         
 17  event_type_2      uint8         
 18  snap_CA           uint8         
 19  snap_TX           uint8         
 20  snap_WI           uint8         
 21  

In [36]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id', 
             "event_name_1", "event_type_1", "event_type_2", 'snap_CA', 'snap_TX', 'snap_WI']
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)

In [37]:
del df
gc.collect()

20

In [38]:
leave_size = 8
params = {
        "objective" : "tweedie",
        'tweedie_variance_power': 1.4,
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
        "nthread" : 10,
        "metric": ["rmse"],
        'verbosity': 20,
        'num_leaves': 2**leave_size-1,
        "min_data_in_leaf": 2**(leave_size + 1)-1,
        "n_estimators": 300
}

In [39]:
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'tweedie',
#     'tweedie_variance_power': 1.1,
#     'metric': 'rmse',
#     'subsample': 0.5,
#     'subsample_freq': 1,
#     'learning_rate': 0.03,
#     'num_leaves': 2**11-1,
#     'min_data_in_leaf': 2**12-1,
#     'feature_fraction': 0.5,
#     'max_bin': 100,
#     'n_estimators': 1400,
#     'boost_from_average': False,
# #     'verbose': -1,
#     'verbosity': 20,
# } 

In [ ]:
%%time

from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

def rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))

m_lgb = lgb.LGBMRegressor(**params)

param_grid = {
    'learning_rate': [0.075, 0.09],
    'n_estimators': [160],
    'num_leaves': [2**ls-1 for ls in [8, 9]],
    "min_data_in_leaf": [2**(ls + 1)-1 for ls in [8, 9]]
}

rmse_score = make_scorer(rmse, greater_is_better = False)
gbm = GridSearchCV(estimator=m_lgb, param_grid=param_grid, cv=3, verbose=3, return_train_score=False, 
                   scoring=rmse_score, n_jobs = 1)

gbm.fit(X=X_train.loc[train_inds], y=y_train.loc[train_inds], 
          eval_set=[(X_train.loc[fake_valid_inds], y_train.loc[fake_valid_inds])],
          eval_names=['sales'], 
          eval_metric=params['metric'],
          verbose=params['verbosity'],
          early_stopping_rounds=100,
          categorical_feature=cat_feats)

print('Best parameters found by grid search are:', gbm.best_params_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] learning_rate=0.075, min_data_in_leaf=511, n_estimators=160, num_leaves=255 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_id', 'dept_id', 'event_name_1', 'event_type_1', 'event_type_2', 'item_id', 'snap_CA', 'snap_TX', 'snap_WI', 'state_id', 'store_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 100 rounds
[20]	sales's rmse: 2.60766
[40]	sales's rmse: 2.09879
[60]	sales's rmse: 2.01809
[80]	sales's rmse: 1.99716
[100]	sales's rmse: 1.98983
[120]	sales's rmse: 1.98741
[140]	sales's rmse: 1.98333
[160]	sales's rmse: 1.97981
Did not meet early stopping. Best iteration is:
[159]	sales's rmse: 1.97979
[CV]  learning_rate=0.075, min_data_in_leaf=511, n_estimators=160, num_leaves=255, score=-2.010, total= 5.3min
[CV] learning_rate=0.075, min_data_in_leaf=511, n_estimators=160, num_leaves=255 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.3min remaining:    0.0s
/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_id', 'dept_id', 'event_name_1', 'event_type_1', 'event_type_2', 'item_id', 'snap_CA', 'snap_TX', 'snap_WI', 'state_id', 'store_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 100 rounds
[20]	sales's rmse: 2.61582
[40]	sales's rmse: 2.10405
[60]	sales's rmse: 2.02077
[80]	sales's rmse: 1.99798
[100]	sales's rmse: 1.99184
[120]	sales's rmse: 1.98858
[140]	sales's rmse: 1.98345
[160]	sales's rmse: 1.98025
Did not meet early stopping. Best iteration is:
[160]	sales's rmse: 1.98025
[CV]  learning_rate=0.075, min_data_in_leaf=511, n_estimators=160, num_leaves=255, score=-2.005, total= 5.4min
[CV] learning_rate=0.075, min_data_in_leaf=511, n_estimators=160, num_leaves=255 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 10.6min remaining:    0.0s
/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_id', 'dept_id', 'event_name_1', 'event_type_1', 'event_type_2', 'item_id', 'snap_CA', 'snap_TX', 'snap_WI', 'state_id', 'store_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


In [ ]:
cv_results = pd.DataFrame(gbm.cv_results_)
cv_results.sort_values(['mean_test_score'])

In [ ]:
cv_results[['param_learning_rate', 'param_num_leaves', 'param_min_data_in_leaf', 'param_n_estimators', 'mean_test_score']]

In [ ]:
class Importance():
    
    def __init__(self, model, eval_metric):
        self.model, self.eval_metric = model, eval_metric
        self.feature_importances = pd.DataFrame(list(zip(X_train.columns, model.feature_importances_)),
                                           columns=['feature', 'importance'])

    def plot_feature_importance(self, drop_null_importance: bool = True, top_n: int = 10):
        """
        Plot default feature importance.

        :param drop_null_importance: drop columns with null feature importance
        :param top_n: show top n columns
        :return:
        """

        top_feats = self.get_top_features(drop_null_importance, top_n)
        feature_importances = self.feature_importances.loc[self.feature_importances['feature'].isin(top_feats)]
        feature_importances['feature'] = feature_importances['feature'].astype(str)
        top_feats = [str(i) for i in top_feats]
        a4_dims = (11.7, 8.27)
        fig, ax = plt.subplots(figsize=a4_dims)
        sns.barplot(data=feature_importances, x='importance', y='feature', orient='h', order=top_feats, ax=ax)
        plt.title('Feature importances')

    def get_top_features(self, drop_null_importance: bool = True, top_n: int = 10):
        """
        Get top features by importance.

        :param drop_null_importance:
        :param top_n:
        :return:
        """
        grouped_feats = self.feature_importances.groupby(['feature'])['importance'].mean()
        if drop_null_importance:
            grouped_feats = grouped_feats[grouped_feats != 0]
        return list(grouped_feats.sort_values(ascending=False).index)[:top_n]
    
    def plot_metric(self):
        """
        Plot training progress.
        Inspired by `plot_metric` from https://lightgbm.readthedocs.io/en/latest/_modules/lightgbm/plotting.html

        :return:
        """
        full_evals_results = pd.DataFrame()
        for model in [self.model]:
            evals_result = pd.DataFrame()
            for k in model.evals_result_.keys():
                evals_result[k] = model.evals_result_[k][self.eval_metric]
            evals_result = evals_result.reset_index().rename(columns={'index': 'iteration'})
            full_evals_results = full_evals_results.append(evals_result)

        full_evals_results = full_evals_results.melt(id_vars=['iteration']).rename(columns={'value': self.eval_metric,
                                                                                            'variable': 'dataset'})
        sns.lineplot(data=full_evals_results, x='iteration', y=self.eval_metric, hue='dataset')
#         categorical_feature  plt.title('Training progress')

In [ ]:
importance = Importance(m_lgb, 'rmse')
importance.plot_feature_importance(top_n=40)

In [ ]:
importance.plot_metric()